Hybrid_model : ConvNext-FPN Mask2Former 만 사용, Gaussian Distance 사용, mAP 0 뜸. F1, precision, recall 은 올라감. 근데 전혀 못 맞춰서 실패

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, LinearLR, SequentialLR
from torch.cuda.amp import GradScaler, autocast

from pathlib import Path
from PIL import Image
import numpy as np
import json
from tqdm import tqdm
from types import SimpleNamespace
import os

# --- 프로젝트 경로 추가 및 모듈 임포트 ---
import sys
sys.path.append('.')

from models.hybrid_model import HybridUnifiedModel
from utils.criterion import SetCriterion
from utils.hungarian_matcher import HungarianMatcher
from torchmetrics.detection import MeanAveragePrecision
from torchmetrics.classification import MulticlassJaccardIndex, MulticlassF1Score, MulticlassPrecision, MulticlassRecall
from pycocotools import mask as mask_utils

# --- 최종 설정 ---
class FinalConfig:
    DATA_ROOT = Path('./data/master_dataset')
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    EPOCHS = 50
    LR = 3e-5
    WEIGHT_DECAY = 1e-4
    BATCH_SIZE = 4
    NUM_WORKERS = 0
    GRADIENT_CLIP_VAL = 1.0
    WARMUP_EPOCHS = 5

    MODEL = SimpleNamespace(
        BACKBONE=SimpleNamespace(NAME='ConvNeXt-Tiny'),
        FPN=SimpleNamespace(OUT_CHANNELS=256),
        HEAD_B=SimpleNamespace(
            FEAT_CHANNELS=256,
            OUT_CHANNELS=256,
            NUM_CLASSES=4,
            QUERIES_PER_CLASS=75,
            DEC_LAYERS=6
        )
    )
    LOSS = SimpleNamespace(
        CLASS_WEIGHTS=[0.5, 1.5, 1.0, 1.3],
        EOS_COEF=0.1
    )
config = FinalConfig()

# --- 데이터셋 클래스 및 로더 ---
class UnifiedDataset(Dataset):
    def __init__(self, image_infos, annotations_map, images_dir, transform=None):
        self.image_infos = image_infos
        self.annotations_map = annotations_map
        self.images_dir = images_dir
        self.transform = transform
        self.cat_map = {1: 0, 2: 1, 3: 2, 4: 3}

    def __len__(self):
        return len(self.image_infos)

    def __getitem__(self, idx):
        img_info = self.image_infos[idx]
        img_id = img_info['id']
        image = Image.open(self.images_dir / img_info['file_name']).convert('RGB')
        original_w, original_h = image.size
        
        target = {}
        masks, labels = [], []
        annotations = self.annotations_map.get(img_id, [])
        for ann in annotations:
            if not ann.get('segmentation') or not ann['segmentation'][0] or len(ann['segmentation'][0]) < 6: continue
            rle = mask_utils.frPyObjects([ann['segmentation'][0]], original_h, original_h)
            mask = mask_utils.decode(rle)
            if mask.ndim == 3: mask = np.max(mask, axis=2)
            masks.append(mask)
            labels.append(self.cat_map[ann['category_id']])

        if self.transform: image = self.transform(image)
        
        target['labels'] = torch.tensor(labels, dtype=torch.int64)
        
        # --- [핵심 수정] 마스크 타입을 다시 float32로 변경 ---
        if masks:
            target['masks'] = torch.from_numpy(np.stack(masks)).to(torch.float32)
        else:
            target['masks'] = torch.zeros((0, original_h, original_w), dtype=torch.float32)
            
        return image, target

def collate_fn(batch): return tuple(zip(*batch))

# --- 데이터 로딩 실행 ---
print("--- Loading Master Dataset ---")
with open(config.DATA_ROOT / 'master_annotations.json', 'r') as f: master_data = json.load(f)
images_info = master_data['images']
annotations_map = {}
for ann in master_data['annotations']:
    img_id = ann['image_id']
    if img_id not in annotations_map: annotations_map[img_id] = []
    annotations_map[img_id].append(ann)

transform = transforms.Compose([
    transforms.Resize((640, 640)), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
full_dataset = UnifiedDataset(images_info, annotations_map, config.DATA_ROOT / 'images', transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS, collate_fn=collate_fn)
print("✅ DataLoaders created!")

# --- 모델, 손실함수, 옵티마이저 초기화 ---
print("--- Initializing Hybrid Unified Model ---")
model = HybridUnifiedModel(config).to(config.DEVICE)
matcher = HungarianMatcher(num_classes=config.MODEL.HEAD_B.NUM_CLASSES)
weight_dict = {'loss_ce': 2.0, 'loss_mask': 5.0, 'loss_dice': 5.0}
criterion = SetCriterion(
    num_classes=config.MODEL.HEAD_B.NUM_CLASSES, matcher=matcher, weight_dict=weight_dict,
    eos_coef=config.LOSS.EOS_COEF, losses=['labels', 'masks'], class_weights=config.LOSS.CLASS_WEIGHTS
).to(config.DEVICE)
optimizer = AdamW(model.parameters(), lr=config.LR, weight_decay=config.WEIGHT_DECAY)
scaler = GradScaler()
warmup_scheduler = LinearLR(optimizer, start_factor=0.1, total_iters=config.WARMUP_EPOCHS)
main_scheduler = CosineAnnealingLR(optimizer, T_max=config.EPOCHS - config.WARMUP_EPOCHS, eta_min=1e-7)
lr_scheduler = SequentialLR(optimizer, schedulers=[warmup_scheduler, main_scheduler], milestones=[config.WARMUP_EPOCHS])
print("✅ Initialization complete.")

--- Loading Master Dataset ---
✅ DataLoaders created!
--- Initializing Hybrid Unified Model ---
✅ Initialization complete.


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13772\2379612393.py:134: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [17]:
def train_epoch(model, criterion, dataloader, optimizer, device, epoch):
    model.train()
    criterion.train()
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{config.EPOCHS} [Train]")
    for images, targets in pbar:
        images = torch.stack(images).to(device)
        targets_gpu = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with autocast():
            outputs = model(images)
            loss_dict = criterion(outputs, targets_gpu)
            weighted_loss = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
        optimizer.zero_grad()
        scaler.scale(weighted_loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), config.GRADIENT_CLIP_VAL)
        scaler.step(optimizer)
        scaler.update()
        pbar.set_postfix({'loss': f'{weighted_loss.item():.4f}'})

def validate(model, criterion, dataloader, device):
    """
    모델 검증 함수
    Args:
        model: 평가할 모델
        criterion: 손실 함수 (matcher 접근용)
        dataloader: 검증 데이터 로더
        device: 디바이스 (cuda/cpu)
    Returns:
        metrics: 평가 지표 딕셔너리
    """
    model.eval()
    
    # --- 평가 지표 객체 초기화 ---
    num_classes = config.MODEL.HEAD_B.NUM_CLASSES
    
    # 1. mAP (Detection/Segmentation 표준)
    map_metric = MeanAveragePrecision(iou_type="segm")
    
    # 2. IoU (Jaccard Index) - 클래스별 평균
    iou_metric = MulticlassJaccardIndex(num_classes=num_classes, average='macro').to(device)
    
    # 3. Precision, Recall, F1-score - 클래스별 평균
    precision_metric = MulticlassPrecision(num_classes=num_classes, average='macro').to(device)
    recall_metric = MulticlassRecall(num_classes=num_classes, average='macro').to(device)
    f1_metric = MulticlassF1Score(num_classes=num_classes, average='macro').to(device)
    
    # 메트릭 업데이트 여부 추적
    metrics_updated = False
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc="[Valid]")
        for images, targets in pbar:
            images = torch.stack(images).to(device)
            
            with autocast():
                outputs = model(images)
            
            # --- mAP 계산용 데이터 준비 ---
            preds_cpu = {k: v.cpu() for k, v in outputs.items() if torch.is_tensor(v)}
            
            preds_for_map = []
            for i in range(len(targets)):
                scores, labels = F.softmax(preds_cpu['pred_logits'][i], dim=-1).max(-1)
                # boolean 마스크를 uint8로 변환
                masks_uint8 = (torch.sigmoid(preds_cpu['pred_masks'][i]) > 0.5).to(torch.uint8)
                preds_for_map.append(dict(masks=masks_uint8, scores=scores, labels=labels))

            targets_for_map = []
            for t in targets:
                target_dict = {
                    'labels': t['labels'].cpu(),
                    # 정답 마스크도 uint8 타입으로 변환
                    'masks': t['masks'].cpu().to(torch.uint8)
                }
                targets_for_map.append(target_dict)

            map_metric.update(preds_for_map, targets_for_map)
            
            # --- IoU, P, R, F1 계산용 데이터 준비 ---
            indices = criterion.matcher(outputs, [{k: v.to(device) for k, v in t.items()} for t in targets])
            
            if any(len(i[0]) > 0 for i in indices):
                idx = criterion._get_src_permutation_idx(indices)
                pred_logits = outputs['pred_logits'][idx]
                target_labels = torch.cat([t["labels"][J] for t, (_, J) in zip(targets, indices)]).to(device)

                # 매칭된 결과로 지표 업데이트
                iou_metric.update(pred_logits, target_labels)
                precision_metric.update(pred_logits, target_labels)
                recall_metric.update(pred_logits, target_labels)
                f1_metric.update(pred_logits, target_labels)
                metrics_updated = True

    # 최종 결과 집계
    try:
        map_results = map_metric.compute()
        map_val = map_results.get('map', torch.tensor(0.0))
        map_50_val = map_results.get('map_50', torch.tensor(0.0))
        
        # tensor인지 확인하고 item() 호출
        if torch.is_tensor(map_val):
            map_val = map_val.item()
        if torch.is_tensor(map_50_val):
            map_50_val = map_50_val.item()
    except:
        print("⚠️ Warning: mAP computation failed")
        map_val = 0.0
        map_50_val = 0.0
    
    # 다른 메트릭들 계산
    if metrics_updated:
        try:
            iou_val = iou_metric.compute().item()
        except:
            iou_val = 0.0
        
        try:
            precision_val = precision_metric.compute().item()
        except:
            precision_val = 0.0
            
        try:
            recall_val = recall_metric.compute().item()
        except:
            recall_val = 0.0
            
        try:
            f1_val = f1_metric.compute().item()
        except:
            f1_val = 0.0
    else:
        # 메트릭이 한 번도 업데이트되지 않은 경우
        print("⚠️ Warning: No valid predictions matched with targets")
        iou_val = precision_val = recall_val = f1_val = 0.0
    
    metrics = {
        'mAP': map_val,
        'mAP_50': map_50_val,
        'iou': iou_val,
        'precision': precision_val,
        'recall': recall_val,
        'f1_score': f1_val
    }
    
    return metrics



In [18]:
# --- 메인 학습 루프 (완전한 버전) ---
print("\n--- 🚀 Starting Hybrid Unified Model Training 🚀 ---")
best_map = 0.0
for epoch in range(config.EPOCHS):
    # 학습
    train_epoch(model, criterion, train_loader, optimizer, config.DEVICE, epoch)
    
    # 검증 - criterion도 전달
    val_metrics = validate(model, criterion, val_loader, config.DEVICE)
    
    # 메트릭 출력
    val_map = val_metrics['mAP']
    val_map_50 = val_metrics['mAP_50']
    
    print(f"\nEpoch {epoch+1}/{config.EPOCHS}:")
    print(f"  Val mAP: {val_map:.4f} | mAP@.50: {val_map_50:.4f}")
    print(f"  IoU: {val_metrics['iou']:.4f} | F1: {val_metrics['f1_score']:.4f}")
    print(f"  Precision: {val_metrics['precision']:.4f} | Recall: {val_metrics['recall']:.4f}")
    
    # 스케줄러 업데이트
    lr_scheduler.step()
    
    # 베스트 모델 저장
    if val_map > best_map:
        best_map = val_map
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_map': best_map,
            'metrics': val_metrics
        }, 'hybrid_unified_best_model.pth')
        print(f"✨ New best model saved with mAP: {best_map:.4f}")
        
print(f"\n--- 🎉 Training Complete ---")
print(f"Best mAP achieved: {best_map:.4f}")


--- 🚀 Starting Hybrid Unified Model Training 🚀 ---


Epoch 1/50 [Train]:   0%|          | 0/655 [00:00<?, ?it/s]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13772\1293238820.py:8: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
[Valid]:   0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13772\1293238820.py:55: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
[Valid]: 100%|██████████| 164/164 [00:26<00:00,  6.24it/s]



Epoch 1/50:
  Val mAP: 0.0000 | mAP@.50: 0.0000
  IoU: 0.2091 | F1: 0.3120
  Precision: 0.5245 | Recall: 0.3580


[Valid]: 100%|██████████| 164/164 [00:27<00:00,  5.90it/s]



Epoch 2/50:
  Val mAP: 0.0000 | mAP@.50: 0.0000
  IoU: 0.1826 | F1: 0.2692
  Precision: 0.5048 | Recall: 0.4049


...


Epoch 48/50:
  Val mAP: 0.0000 | mAP@.50: 0.0000
  IoU: 0.7470 | F1: 0.8474
  Precision: 0.8647 | Recall: 0.8369


[Valid]: 100%|██████████| 164/164 [00:26<00:00,  6.15it/s]



Epoch 49/50:
  Val mAP: 0.0000 | mAP@.50: 0.0000
  IoU: 0.7545 | F1: 0.8522
  Precision: 0.8670 | Recall: 0.8433


[Valid]: 100%|██████████| 164/164 [00:26<00:00,  6.19it/s]



Epoch 50/50:
  Val mAP: 0.0000 | mAP@.50: 0.0000
  IoU: 0.7533 | F1: 0.8514
  Precision: 0.8700 | Recall: 0.8407

--- 🎉 Training Complete ---
Best mAP achieved: 0.0000


In [21]:
print("--- 🚀 Starting Single Batch Overfit Test ---")

# 1. 학습 데이터 로더에서 딱 한 개의 배치만 가져오기
try:
    single_batch = next(iter(train_loader))
    print("✅ Single batch loaded successfully.")
except Exception as e:
    print(f"❌ Failed to load a batch: {e}")

if 'single_batch' in locals():
    images, targets = single_batch
    images = torch.stack(images).to(config.DEVICE)
    targets_gpu = [{k: v.to(config.DEVICE) for k, v in t.items()} for t in targets]

    # 2. 모델과 옵티마이저를 새로 초기화 (깨끗한 상태에서 시작)
    model_test = HybridUnifiedModel(config).to(config.DEVICE)
    optimizer_test = AdamW(model_test.parameters(), lr=config.LR)
    criterion_test = criterion.to(config.DEVICE)
    scaler_test = GradScaler()
    model_test.train()
    
    print("\n--- Starting to overfit on the single batch for 200 iterations ---")
    # 3. 동일한 배치로 200번 학습 시도
    for i in tqdm(range(200)):
        with autocast():
            outputs = model_test(images)
            loss_dict = criterion_test(outputs, targets_gpu)
            weighted_loss = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

        optimizer_test.zero_grad()
        scaler_test.scale(weighted_loss).backward()
        scaler_test.step(optimizer_test)
        scaler_test.update()
        
        if (i + 1) % 10 == 0:
            print(f"Iteration {i+1}/200 -> Loss: {weighted_loss.item():.4f}")

--- 🚀 Starting Single Batch Overfit Test ---
✅ Single batch loaded successfully.


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13772\547243036.py:19: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler_test = GradScaler()



--- Starting to overfit on the single batch for 200 iterations ---


  0%|          | 0/200 [00:00<?, ?it/s]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13772\547243036.py:25: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
  5%|▌         | 10/200 [00:05<01:48,  1.75it/s]

Iteration 10/200 -> Loss: 10.7047


...

Iteration 180/200 -> Loss: 7.2570


 95%|█████████▌| 190/200 [01:48<00:05,  1.76it/s]

Iteration 190/200 -> Loss: 8.6877


100%|██████████| 200/200 [01:53<00:00,  1.75it/s]

Iteration 200/200 -> Loss: 8.6085


Unified_model : ConvNext-FPN Mask2Former 만 사용, Gaussian Distance 사용.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, LinearLR, SequentialLR
from torch.cuda.amp import GradScaler, autocast

from pathlib import Path
from PIL import Image
import numpy as np
import json
from tqdm import tqdm
from types import SimpleNamespace
import os

# --- 프로젝트 경로 추가 및 모듈 임포트 ---
import sys
sys.path.append('.')

from models.unified_model import UnifiedModel
from utils.criterion import SetCriterion
from utils.hungarian_matcher import HungarianMatcher
from torchmetrics.detection import MeanAveragePrecision
from torchmetrics.classification import MulticlassJaccardIndex, MulticlassF1Score, MulticlassPrecision, MulticlassRecall
from pycocotools import mask as mask_utils

print("--- 🚀 Starting Single Batch Overfit Test (Simple Model) ---")

# 1. 학습 데이터 로더에서 딱 한 개의 배치만 가져오기
try:
    single_batch = next(iter(train_loader))
    print("✅ Single batch loaded successfully.")
except Exception as e:
    print(f"❌ Failed to load a batch: {e}")

if 'single_batch' in locals():
    images, targets = single_batch
    images = torch.stack(images).to(config.DEVICE)
    targets_gpu = [{k: v.to(config.DEVICE) for k, v in t.items()} for t in targets]

    # 2. 모델과 옵티마이저를 새로 초기화 (깨끗한 상태에서 시작)
    model_test = UnifiedModel(config).to(config.DEVICE)
    optimizer_test = AdamW(model_test.parameters(), lr=config.LR)
    criterion_test = criterion.to(config.DEVICE)
    scaler_test = GradScaler()
    model_test.train()
    
    print("\n--- Starting to overfit on the single batch for 200 iterations ---")
    # 3. 동일한 배치로 200번 학습 시도
    for i in tqdm(range(200)):
        with autocast():
            outputs = model_test(images)
            loss_dict = criterion_test(outputs, targets_gpu)
            weighted_loss = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

        optimizer_test.zero_grad()
        scaler_test.scale(weighted_loss).backward()
        scaler_test.step(optimizer_test)
        scaler_test.update()
        
        if (i + 1) % 10 == 0:
            print(f"Iteration {i+1}/200 -> Loss: {weighted_loss.item():.4f}")

--- Loading the best model and visualizing predictions ---


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13772\2759810527.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('hybrid_unified

FileNotFoundError: [Errno 2] No such file or directory: 'hybrid_unified_best_model.pth'